<a href="https://colab.research.google.com/github/sanjitk7/textSummarisationGithubIssues/blob/master/textSummarisation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import tensorflow as tf
import pandas as pd

In [57]:
%tensorflow_version 1.x

TensorFlow is already loaded. Please restart the runtime to change versions.


In [12]:
import logging
import glob
from sklearn.model_selection import train_test_split
pd.set_option('display.max_colwidth', 500)
logger = logging.getLogger()
logger.setLevel(logging.WARNING)
print(tf.__version__)

2.2.0


In [11]:
df = pd.read_csv("/content/drive/My Drive/natural-language-processing/github_issues.csv")
df

,issue_url,issue_title,body
0,"""https://github.com/zhangyuanwei/node-images/i...",can't load the addon. issue to: https://github...,can't load the addon. issue to: https://github...
1,"""https://github.com/Microsoft/pxt/issues/2543""",hcl accessibility a11yblocking a11ymas mas4.2....,user experience: user who depends on screen re...
2,"""https://github.com/MatisiekPL/Czekolada/issue...",issue 1265: issue 1264: issue 1261: issue 1260...,┆attachments: <a href= https:& x2f;& x2f;githu...
3,"""https://github.com/MatisiekPL/Czekolada/issue...",issue 1266: issue 1263: issue 1262: issue 1259...,gitlo = github x trello\n---\nthis board is no...
4,"""https://github.com/MatisiekPL/Czekolada/issue...",issue 1288: issue 1285: issue 1284: issue 1281...,┆attachments: <a href= https:& x2f;& x2f;githu...
...,...,...,...
5332148,"""https://github.com/bayborodin/ror-full-3/issu...","создать модуль instancecounter, содержащий сле...","методы класса: - instances, который возвращает..."
5332149,"""https://github.com/eclipse/paho.mqtt.java/iss...",at org.eclipse.paho.client.mqttv3.internal.cli...,- bug exists release version 1.1.0 master bran...
5332150,"""https://github.com/rzwitserloot/lombok/issues...",java.lang.linkageerror: loader constraint viol...,java.lang.linkageerror: loader constraint viol...
5332151,"""https://github.com/Gizra/productivity/issues/...",node : pdoexception: sqlstate 40001 : serializ...,view details in rollbar: https://rollbar.com/b...


In [27]:
#read in data sample 2M rows (for speed of tutorial)
traindf, testdf = train_test_split(df.sample(n=200), 
                                   test_size=.10)


#print out stats about shape of data
print(f'Train: {traindf.shape[0]:,} rows {traindf.shape[1]:,} columns')
print(f'Test: {testdf.shape[0]:,} rows {testdf.shape[1]:,} columns')

# preview data
traindf.head(3)

Train: 180 rows 3 columns
Test: 20 rows 3 columns


,issue_url,issue_title,body
1332149,"""https://github.com/fstpackage/fst/issues/102""",your dataset needs at least one column.,"i was interested to use this package to save large datasets. however, i am getting this error when trying to create a simple benchmark. r df <- data.frame matrix runif 256 65536 , nrow = 256 file <- tempfile fileext = .fst system.time fst::write.fst df, file > error in fststore path, x, as.integer compress : your dataset needs at least one column. > timing stopped at: 0.044 0 0.045"
3387148,"""https://github.com/elixir-lang/elixir/issues/6172""",elixir-lang.org blocked in russia,"blocking date: may 10, 2016 status url: in registry domain: k-shkaf.ru ip: 104.24.104.149 | 104.24.105.149 | 104.28.10.110 | 104.28.11.110 ip elixir-lang.org 104.28.10.110"
2711216,"""https://github.com/fuji97/pokemon-xenoverse-tracker/issues/139""",mosse non prioritarie,mi capita spesso che un pkmn avversario più lento di me attacchi prima senza usare mosse prioritarie o rapidartigli


In [28]:
train_body_raw = traindf.body.tolist()
train_title_raw = traindf.issue_title.tolist()
#preview output of first element
train_body_raw[0]

'i was interested to use this package to save large datasets. however, i am getting this error when trying to create a simple benchmark. r df <- data.frame matrix runif 256 65536 , nrow = 256 file <- tempfile fileext = .fst system.time fst::write.fst df, file > error in fststore path, x, as.integer compress : your dataset needs at least one column. > timing stopped at: 0.044 0 0.045'

In [22]:
!pip install -q ktext

In [23]:
%reload_ext autoreload
%autoreload 2
from ktext.preprocess import processor

In [29]:
%%time
# Clean, tokenize, and apply padding / truncating such that each document length = 70
#  also, retain only the top 8,000 words in the vocabulary and set the remaining words
#  to 1 which will become common index for rare words 
body_pp = processor(keep_n=8000, padding_maxlen=70)
train_body_vecs = body_pp.fit_transform(train_body_raw)

CPU times: user 38.5 ms, sys: 177 ms, total: 216 ms
Wall time: 576 ms


In [30]:
print('\noriginal string:\n', train_body_raw[0], '\n')
print('after pre-processing:\n', train_body_vecs[0], '\n')


original string:
 i was interested to use this package to save large datasets. however, i am getting this error when trying to create a simple benchmark. r df <- data.frame matrix runif 256 65536 , nrow = 256 file <- tempfile fileext = .fst system.time fst::write.fst df, file > error in fststore path, x, as.integer compress : your dataset needs at least one column. > timing stopped at: 0.044 0 0.045 

after pre-processing:
 [   0    0    0    0    6   86 1403    4   41   12   92    4  176  877
 1404  626    6  177  475   12   24   21  201    4   87    5  178 1405
  266  878   51  627  389 1406    2    2 1407    2   29 1408 1409  628
  202  179  628  203  628  878   29   24    9 1410  390  267   20  391
 1411   66 1412  180   30  879   57  629 1413 1414   30    2    2    2] 



In [31]:
# Instantiate a text processor for the titles, with some different parameters
#  append_indicators = True appends the tokens '_start_' and '_end_' to each
#                      document
#  padding = 'post' means that zero padding is appended to the end of the 
#             of the document (as opposed to the default which is 'pre')
title_pp = processor(append_indicators=True, keep_n=4500, 
                     padding_maxlen=12, padding ='post')

# process the title data
train_title_vecs = title_pp.fit_transform(train_title_raw)

In [32]:
print('\noriginal string:\n', train_title_raw[0])
print('after pre-processing:\n', train_title_vecs[0])


original string:
 your dataset needs at least one column.
after pre-processing:
 [  2 151 152  70  44 153  71 154   3   0   0   0]


In [33]:
import dill as dpickle
import numpy as np

# Save the preprocessor
with open('body_pp.dpkl', 'wb') as f:
    dpickle.dump(body_pp, f)

with open('title_pp.dpkl', 'wb') as f:
    dpickle.dump(title_pp, f)

# Save the processed data
np.save('train_title_vecs.npy', train_title_vecs)
np.save('train_body_vecs.npy', train_body_vecs)

In [48]:
!pip install -q annoy

     |████████████████████████████████| 645kB 2.6MB/s 


In [49]:
!cp "/content/drive/My Drive/natural-language-processing/seq2seq_utils.py" .
from seq2seq_utils import load_decoder_inputs, load_encoder_inputs, load_text_processor

In [50]:
encoder_input_data, doc_length = load_encoder_inputs('train_body_vecs.npy')
decoder_input_data, decoder_target_data = load_decoder_inputs('train_title_vecs.npy')

Shape of encoder input: (180, 70)
Shape of decoder input: (180, 11)
Shape of decoder target: (180, 11)


In [51]:
num_encoder_tokens, body_pp = load_text_processor('body_pp.dpkl')
num_decoder_tokens, title_pp = load_text_processor('title_pp.dpkl')

Size of vocabulary for body_pp.dpkl: 3,156
Size of vocabulary for title_pp.dpkl: 760


In [52]:
%matplotlib inline
from keras.models import Model
from keras.layers import Input, LSTM, GRU, Dense, Embedding, Bidirectional, BatchNormalization
from keras import optimizers

In [54]:
#arbitrarly set latent dimension for embedding and hidden units
latent_dim = 300

##### Define Model Architecture ######

########################
#### Encoder Model ####
encoder_inputs = Input(shape=(doc_length,), name='Encoder-Input')

# Word embeding for encoder (ex: Issue Body)
x = Embedding(num_encoder_tokens, latent_dim, name='Body-Word-Embedding', mask_zero=False)(encoder_inputs)
x = BatchNormalization(name='Encoder-Batchnorm-1')(x)

# Intermediate GRU layer (optional)
#x = GRU(latent_dim, name='Encoder-Intermediate-GRU', return_sequences=True)(x)
#x = BatchNormalization(name='Encoder-Batchnorm-2')(x)

# We do not need the `encoder_output` just the hidden state.
_, state_h = GRU(latent_dim, return_state=True, name='Encoder-Last-GRU')(x)

# Encapsulate the encoder as a separate entity so we can just 
#  encode without decoding if we want to.
encoder_model = Model(inputs=encoder_inputs, outputs=state_h, name='Encoder-Model')

seq2seq_encoder_out = encoder_model(encoder_inputs)

########################
#### Decoder Model ####
decoder_inputs = Input(shape=(None,), name='Decoder-Input')  # for teacher forcing

# Word Embedding For Decoder (ex: Issue Titles)
dec_emb = Embedding(num_decoder_tokens, latent_dim, name='Decoder-Word-Embedding', mask_zero=False)(decoder_inputs)
dec_bn = BatchNormalization(name='Decoder-Batchnorm-1')(dec_emb)

# Set up the decoder, using `decoder_state_input` as initial state.
decoder_gru = GRU(latent_dim, return_state=True, return_sequences=True, name='Decoder-GRU')
decoder_gru_output, _ = decoder_gru(dec_bn, initial_state=seq2seq_encoder_out)
x = BatchNormalization(name='Decoder-Batchnorm-2')(decoder_gru_output)

# Dense layer for prediction
decoder_dense = Dense(num_decoder_tokens, activation='softmax', name='Final-Output-Dense')
decoder_outputs = decoder_dense(x)

########################
#### Seq2Seq Model ####

#seq2seq_decoder_out = decoder_model([decoder_inputs, seq2seq_encoder_out])
seq2seq_Model = Model([encoder_inputs, decoder_inputs], decoder_outputs)


seq2seq_Model.compile(optimizer=optimizers.Nadam(lr=0.001), loss='sparse_categorical_crossentropy')

AttributeError: ignored